In [20]:
import os

In [21]:
%pwd

'd:\\work_projects\\text_summarizer_project\\research'

In [22]:
os.chdir("../")

In [23]:
%pwd

'd:\\work_projects\\text_summarizer_project'

In [24]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [25]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [26]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [27]:
import os
import urllib.request as request
import zipfile
from text_summarizer.loggin import logger
from text_summarizer.utils.common import get_size

In [28]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
            zip_file_path: str,
            Extracts the zip file into data directory
            Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    

In [29]:
from pathlib import Path
from text_summarizer.constants import CONFIG_FILE_PATH

print("Resolved CONFIG_FILE_PATH:", CONFIG_FILE_PATH)
print("Absolute path:", CONFIG_FILE_PATH.resolve())
print("Exists:", CONFIG_FILE_PATH.exists())

Resolved CONFIG_FILE_PATH: config\config.yaml
Absolute path: D:\work_projects\text_summarizer_project\config\config.yaml
Exists: True


In [30]:
import traceback

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    
    print("✅ Config loaded successfully")
    
    data_ingestion = DataIngestion(config=data_ingestion_config)
    print("✅ DataIngestion class initialized")
    
    data_ingestion.download_file()
    print("✅ Download complete")
    
    data_ingestion.extract_zip_file()
    print("✅ Extraction complete")

except Exception as e:
    print("❌ An error occurred during data  s ingestion:")
    traceback.print_exc()


[2025-04-06 16:22:22,280:  INFO: common - yaml file: <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='utf-8'> loaded successfully]
[2025-04-06 16:22:22,291:  INFO: common - yaml file: <_io.TextIOWrapper name='params.yaml' mode='r' encoding='utf-8'> loaded successfully]
[2025-04-06 16:22:22,293:  INFO: common - Created directory at: artifacts]
[2025-04-06 16:22:22,295:  INFO: common - Created directory at: artifacts/data_ingestion]
✅ Config loaded successfully
✅ DataIngestion class initialized
[2025-04-06 16:22:23,804:  INFO: 318965258 - artifacts/data_ingestion/data.zip download! with following info Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=b